In [ ]:
import cv2
import time
import imutils
import copy
import numpy as np
import matplotlib.pyplot as plt
from skimage import img_as_uint
from skimage.filters import roberts
%matplotlib inline

In [ ]:
def detect_upper_body(img):
    img_copy = np.copy(img)
    img_copy = cv2.cvtColor(img_copy, cv2.COLOR_RGB2GRAY)
    upper_body_cascade = cv2.CascadeClassifier('descriptors/upper_mcs_body.xml')
    rectangles = upper_body_cascade.detectMultiScale(img_copy)
    rectangle_list = sorted(rectangles, key=lambda x: x[2] * x[3], reverse=True)
    x, y, w, h =  rectangle_list[0] if rectangle_list else [0,0,0,0]
    cv2.rectangle(img_copy, (x,y), (x+w,y+h),(0,255,0),2)
    image_cut = img_copy[y:y+h, x:x+w]
    plt.imshow(image_cut, cmap="gray")
    return image_cut

def find_contour(gray):
    lower = 80
    upper = 140
    gray = cv2.GaussianBlur(gray, (7, 7), 0)
    mask = cv2.inRange(gray, lower, upper)
    thresh = cv2.bitwise_not(gray, upper_body_image, mask=mask)
    thresh = cv2.threshold(thresh, 45, 255, cv2.THRESH_BINARY_INV + cv2.THRESH_OTSU)[1]
    thresh = cv2.erode(thresh, None, iterations=2)
    thresh = cv2.dilate(thresh, None, iterations=2)
    plt.imshow(thresh, cmap="gray")
    cnts = cv2.findContours(thresh.copy(), cv2.RETR_EXTERNAL, cv2.CHAIN_APPROX_SIMPLE)
    cnts = imutils.grab_contours(cnts)
    return cnts

def preprocess(img):
    edges = roberts(img)
    plt.imshow(edges, cmap="gray")
    dilation = cv2.dilate(edges, None, iterations=2)
    plt.imshow(dilation, cmap="gray")
    return edges

In [ ]:
plt.rcParams['figure.figsize'] = [14.0, 7.0]

image = cv2.imread("images/cora.jpg")
image_copy = np.copy(image)
# image_copy = cv2.resize(image_copy, (0,0), fx=0.1, fy=0.1)
image_copy = cv2.cvtColor(image_copy, cv2.COLOR_BGR2RGB)
plt.imshow(image_copy)

In [ ]:
upper_body_image = detect_upper_body(image_copy)

In [ ]:
cnt = find_contour(upper_body_image)

In [ ]:
maxLeft = tuple(cnt[0][cnt[0][:,:,0].argmin()][0])
maxRight = tuple(cnt[0][cnt[0][:,:,0].argmax()][0])

for c in cnt:
    leftmost = tuple(c[c[:,:,0].argmin()][0])
    rightmost = tuple(c[c[:,:,0].argmax()][0])
    if leftmost < maxLeft:
        maxLeft = leftmost
    if rightmost > maxRight:
        maxRight = rightmost

distance = np.sqrt( (maxRight[0] - maxLeft[0])**2 + (maxRight[1] - maxLeft[1])**2 )
print(f"Left: {maxLeft} | Right: {maxRight} | Distance: {distance}")

In [ ]:
contour_lines = cv2.drawContours(upper_body_image, cnt, -1, (204, 255, 0), 3)
contour_lines = cv2.circle(contour_lines, maxLeft, 25, (0, 0, 255), -1)
contour_lines = cv2.circle(contour_lines, maxRight, 25, (0, 0, 255), -1)

In [ ]:
plt.imshow(contour_lines, cmap="gray")

In [ ]:
plt.imshow(contour_lines)